In [ ]:
#install required libraries

!pip install sqlachemy 
!pip install ipython-sql
! pip install pymysql

ERROR: Could not find a version that satisfies the requirement sqlachemy (from versions: none)
ERROR: No matching distribution found for sqlachemy


In [16]:
import sqlalchemy
from sqlalchemy import create_engine
import pandas as pd
import sqlite3

In [47]:
pip install --upgrade prettytable


Note: you may need to restart the kernel to use updated packages.


In [48]:
#connect sqlite database(it will be created if it is not existed)
%reload_ext sql
%config SqlMagic.style = 'ascii'

engine = create_engine("sqlite:///hr.db")
%config SQL.conn_name = 'engine'

conn = sqlite3.connect('hr.db')
cursor = conn.cursor()

%sql sqlite:///hr.db

#create tables
cursor.execute('DROP TABLE IF EXISTS hr;')
cursor.execute('DROP TABLE IF EXISTS departments;')
cursor.execute('DROP TABLE IF EXISTS locations;')
cursor.execute('DROP TABLE IF EXISTS education_levels;')
cursor.execute('DROP TABLE IF EXISTS job_titles;')
cursor.execute('DROP TABLE IF EXISTS managers;')
cursor.execute('DROP TABLE IF EXISTS employees;')


cursor.execute('''
CREATE TABLE hr (
    emp_id VARCHAR(10) ,
    emp_nm VARCHAR(100),
    email VARCHAR(100),
    hire_dt DATE,
    job_title VARCHAR(100),
    salary DECIMAL(10, 2),
    department VARCHAR(100),
    manager VARCHAR(100),
    start_dt DATE,
    end_dt DATE,  -- Nullable field
    location VARCHAR(100),
    address VARCHAR(200),
    city VARCHAR(100),
    state CHAR(2),
    education_level VARCHAR(50)
);
               ''')



cursor.execute('''
CREATE TABLE departments (
    dept_id SERIAL PRIMARY KEY,
    dept_name VARCHAR(100) UNIQUE
);

               ''')


cursor.execute('''
CREATE TABLE locations (
    location_id SERIAL PRIMARY KEY,
    location_name VARCHAR(100),
    address VARCHAR(200),
    city VARCHAR(100),
    state CHAR(20));
               ''')

cursor.execute('''
CREATE TABLE education_levels (
    edu_level_id SERIAL PRIMARY KEY,
    edu_level_name VARCHAR(50) UNIQUE
);

               ''')

cursor.execute('''
CREATE TABLE job_titles (
    job_id SERIAL PRIMARY KEY,
    job_title VARCHAR(100) UNIQUE
);

               ''')

cursor.execute('''
CREATE TABLE managers (
    manager_id SERIAL PRIMARY KEY,
    manager_name VARCHAR(100)
);

               ''')

cursor.execute('''
CREATE TABLE employees (
    emp_id VARCHAR(10) PRIMARY KEY,
    emp_nm VARCHAR(100),
    email VARCHAR(100),
    hire_dt DATE,
    salary DECIMAL(10, 2),
    start_dt DATE,
    end_dt DATE,
    dept_id INT REFERENCES departments(dept_id),
    job_id INT REFERENCES job_titles(job_id),
    manager_id INT REFERENCES managers(manager_id),
    location_id INT REFERENCES locations(location_id),
    edu_level_id INT REFERENCES education_levels(edu_level_id)
);

               ''')


In [49]:
#Function to load external data into hr table
def load_data_to_table(path , table_name):
    df = pd.read_excel(path)
    
    # Rename columns to match SQLite schema
    df.columns = [col.strip().lower().replace(" ", "_") for col in df.columns]

    #then load records
    df.to_sql(table_name , conn , if_exists='append' , index=False)
    print(f'Data Loaded Successfully to table {table_name}')

load_data_to_table(r'E:\hr\hr-dataset.xlsx' , 'hr')

Data Loaded Successfully to table hr


In [ ]:
cursor.execute('''
INSERT INTO departments (dept_name)
SELECT DISTINCT department
FROM hr
WHERE department IS NOT NULL;

               ''')

cursor.execute('''
INSERT INTO locations (location_name, address, city, state)
SELECT DISTINCT location, address, city, state
FROM hr
WHERE location IS NOT NULL;

               ''')

cursor.execute('''
INSERT INTO job_titles (job_title)
SELECT DISTINCT job_title
FROM hr
WHERE job_title IS NOT NULL;

               ''')

cursor.execute('''
INSERT INTO managers (manager_name)
SELECT DISTINCT manager
FROM hr
WHERE manager IS NOT NULL;

               ''')

cursor.execute('''
INSERT INTO education_levels (edu_level_name)
SELECT DISTINCT education_level
FROM hr
WHERE education_level IS NOT NULL;

               ''')

cursor.execute('''
INSERT OR IGNORE INTO employees (
    emp_id, emp_nm, email, hire_dt, salary, start_dt, end_dt,
    dept_id, job_id, manager_id, location_id, edu_level_id
)
SELECT 
    h.emp_id,
    h.emp_nm,
    h.email,
    h.hire_dt,
    h.salary,
    h.start_dt,
    h.end_dt,
    d.dept_id,
    j.job_id,
    m.manager_id,
    l.location_id,
    e.edu_level_id
FROM hr h
JOIN departments d ON h.department = d.dept_name
JOIN job_titles j ON h.job_title = j.job_title
JOIN managers m ON h.manager = m.manager_name
JOIN locations l ON h.location = l.location_name AND h.address = l.address AND h.city = l.city AND h.state = l.state
JOIN education_levels e ON h.education_level = e.edu_level_name;


               ''')